# Cross-validation

With the code developed so far, it is possible to train an ANN and provide an estimate of the results it would offer in its real execution (with unseen patterns, represented by a test set). However, in this last aspect there are two factors to consider, as a consequence of the non-deterministic nature of the process we are following:

- The partitioning of the set of patterns into training/test is random (hold out), and is therefore overly dependent on good or bad luck in choosing training and test patterns.
- ANN training is not deterministic, as the initialisation of the weights is random. As before, it is too dependent on good or bad luck to start the training at a good or bad starting point.

For these two reasons, the test result of a single training is not significant when assessing the goodness of fit of the model in the presence of unseen patterns. To solve this problem, the experiment is repeated several times and the results are averaged. This can be implemented in a simple way by means of a loop; however, it is necessary to do this in an orderly way as there are two different sources of randomness.

Firstly, to minimise the randomness due to the partitioning of the data set, it is necessary to have a method that ensures that each data is used for training at least once, and for testing at least once. The most commonly used method is cross-validation. In this method, the data set is split into k disjoint subsets and k experiments are performed. In the k-th experiment, the k subset is separated for testing, and the remaining k-1 substes are used for training, performing a k-fold cross-validation. A common value is k=10, which gives a 10-fold cross-validation. Finally, the test value corresponding to the appropriate metric will be the average value of the values of the k experiments.

A widely used variant of cross-validation is stratified cross-validation. In this case, each subset is created in such a way as to keep the proportion of patterns of each class the same (or similar) as in the original dataset. This is particularly used when the data set is imbalanced.

It is usual to save not only the mean, but also the k values, in order to subsequently perform a paired hypothesis test with another model. To do this, it is necessary that both models have been trained using the same training and test sets.

This way of evaluating the model is often considered to be slightly pessimistic, i.e. the results obtained in tests are slightly worse than those that would be obtained from real training with all available data. In a hold out experiment, as mentioned above, several data are separated for testing. This means that the model is trained with less data than is available, and that by chance the data separated for testing can be of great importance (especially if there is little data). For this reason, when training with less data and possibly no "important" data, hold out is considered a pessimistic assessment. In the same way, cross-validation also separates data for testing, so it does not train on all available data, and is therefore also pessimistic. However, it is guaranteed that all data are used at least once in training and once in testing, thus trying to minimise the impact of chance in separating data, so it is considered only a slightly pessimistic evaluation.

Doing this is as simple as splitting the data set and performing a loop with k iterations in which at the k-th iteration a model is trained and evaluated with the corresponding sets. However, if the model is not deterministic, the result obtained at the k-th iteration will not be meaningful, since it is again dependent on chance. In this case, what needs to be done is a second nested loop within iteration k in which the model is repeatedly trained, and finally an average of the results is made to finally output the result of iteration k. The number of trainings must be high for the average results to be really significant, at least 50 trainings.

### Question 5.1

> ❓ If this second loop is performed with a deterministic model, what will be the standard deviation of the test results obtained? Is there a difference between performing this second loop and averaging the results, or doing a single training?

`Answer here`

In this way, it is possible to evaluate a model together with its hyperparameters in solving a problem. A very common situation is to compare several models (or the same model with different hyperparameters), for which this scheme has to be applied with an important caveat: the sets used in the cross-validation must be the same for each model. Since the distribution of patterns in different sets is random, having the same subsets in different runs is achieved by setting the random seed at the beginning of the program to be executed. Setting the random seed not only allows the same subsets to be generated, but is also important in order to be able to repeat the results in different runs.

It is also important to bear in mind that this methodology allows estimating the real performance of a model (although slightly pessimistic). The final model that would be used in production would be the result of training it with all the available patterns, since, as seen in the theory class, and very generally speaking, the more patterns you train with, the better the model will be.

Another commonly requested task is to obtain a **confusion matrix on the test set** as a result of performing **cross-validation**.

In general, this is **not directly possible**, because multiple neural networks are trained for each fold.  
If only a single model were trained per fold (as will be the case in the next exercise), it would be possible to compute a confusion matrix for each test set and then sum all of them to obtain a **global test confusion matrix**.

However, since multiple models are trained in each fold here, we use an alternative approach:

- For each fold, a **confusion matrix is computed for every execution**.
- These matrices are then **averaged within each fold**.
- Finally, the averaged matrices from each fold are **summed** to produce a **global confusion matrix**.

---

### ⚠️ Important considerations

- This global confusion matrix **does not represent the result of any single model**.  
  Therefore, calling it a "confusion matrix" can be misleading — but it is still useful to analyze how instances are distributed in the test sets.

- For the same reason, the **metrics derived from this matrix** will generally **not match** the metric values returned by the function (which are averages of the metric scores for each fold).  
  These matrices are meant to provide **a visual and aggregated summary**, while the true evaluation metrics are the ones computed per fold.

- As a result of averaging the confusion matrices, the **final confusion matrix will contain real numbers**, not integers.  
  While this is technically not correct, it is acceptable to include it in a report — as long as the nature of the matrix is clearly explained.

In this assignment, you are asked to:

1. Develop a function called `crossvalidation` that receives a value `N` (equal to the number of patterns), and a value `k` (number of subsets into which the dataset is to be split), and returns a vector of length N, where each element indicates in which subset that pattern should be included.

    To perform this function, one possibility is to perform the following steps:
    
    - Create a vector with k sorted elements, from 1 to k.
    - Create a new vector with repetitions of the previous vector until its length is greater than or equal to N. The functions `repeat` and `ceil` can be used for this purpose.
    - Take the first N values of this vector.
    - Shuffle this vector (using the function `shuffle!` and return it. To use this function, the module `Random` should be loaded.
    
    This function should return a vector of **integer values**, with a length equal to **N**. No loop should be used in this function.

In [ ]:
function crossvalidation(N::Int64, k::Int64)
    #TODO

2. Develop a new function called `crossvalidation` that receives:

- A vector `targets::AbstractArray{Bool,1}` containing the desired outputs (ground truth labels),
- An integer `k`, which is the number of folds for the partitioning.

The function must return a vector of **integer values** of length **N** (where `N` is the number of elements in `targets`), indicating to which subset (fold) each instance belongs.

The partitioning must be **stratified**, meaning that **the distribution of positive and negative instances is preserved across folds**. 
Follow these steps:

    1. Create a vector of indices with as many elements as there are rows in `targets`.
    2. Call the previously developed `crossvalidation` function, passing the **number of positive instances** and the value of `k`.
    3. Assign the result of the previous step to the positions of the index vector that correspond to **positive instances**.
  ### Question 5.2
  > ❓ Could you combine steps 2 and 3 into a single line? (This is not required, but worth considering.)

`Answer here`

    4. Repeat a similar operation for **negative instances**.
    5. Return the index vector.

Loops are **not allowed** in the implementation of this function. The function must return a **vector of integers** of length **N**.

In [ ]:
function crossvalidation(targets::AbstractArray{Bool,1}, k::Int64)
    #TODO

3. Develop a new function called `crossvalidation` that receives:

- A matrix `targets::AbstractArray{Bool,2}`, where each row corresponds to one instance and each column to one class (multilabel format),
- An integer `k`, indicating the number of folds (subsets) to divide the dataset into.

The function must return a vector of integers of length `N` (where `N` is the number of rows in `targets`).  
Each element in the vector indicates the subset (fold) that the corresponding instance should be assigned to.

The partitioning must be **stratified**, meaning that the distribution of **each class across the folds is preserved**.
Suggested Steps:

    1. Create an index vector with as many elements as rows in `targets`.

    2. Use a loop that **iterates over the classes** (i.e., the columns of `targets`).  
     For each class:

       i. Count the number of instances that belong to that class using `sum(targets[:, i])`.  
       ii. Call the previously defined `crossvalidation` function with the number of instances and `k`.  
       iii. Assign the result to the corresponding positions in the index vector where `targets[:, i]` is `true`.

   ### Question 5.3
   > ❓ Could you combine these three operations into a single line inside the loop? (Optional exercise)

`Answer here`

    3. Return the index vector.

  #### Important considerations

   - This function is allowed to use **a single loop** that iterates over the classes.
   - The result must be a vector of **integer values** with length equal to the number of instances `N`.

  #### Class support per fold

  It is essential to ensure that **each class has at least `k` instances**.

  > A typical value is `k = 10`.

### Question 5.4
> ❓ What would happen if a class has fewer than `k` instances?

`Answer here`

### Question 5.5
> ❓ How would this affect the calculation of metrics such as sensitivity or specificity?

`Answer here`

If, for any reason, it is not possible to guarantee at least `k` samples per class, you may consider **reducing the value of `k`**.  
In this case, consult your instructor to evaluate the implications and whether the trained models would still yield meaningful results.

In [ ]:
function crossvalidation(targets::AbstractArray{Bool,2}, k::Int64)
    #TODO

4. Implement a final version of the `crossvalidation` function, where:

- The first argument is `targets::AbstractArray{<:Any,1}`, a vector containing **heterogeneous elements** (e.g., strings, symbols, integers, etc.),
- The second argument is the usual integer `k` (number of folds),
- The output is a vector of **integer values**, one per instance, indicating the fold assignment.

To implement this function, follow these steps:

   1. Call the `oneHotEncoding` function, passing `targets` as the input.  
      This will convert the labels into a binary matrix (multilabel format).
   2. Call the **previous version** of the `crossvalidation` function, passing the encoded matrix and the value `k`.

   > **No loops are allowed** in this function.
   > The returned vector must contain integers and have the same length as the input.

In [ ]:
function crossvalidation(targets::AbstractArray{<:Any,1}, k::Int64)
    #TODO

5. Finally, implement a function called `ANNCrossValidation` that trains an artificial neural network (ANN) multiple times following a **cross-validation strategy**, as described in this exercise.

 This function will receive both the usual inputs for training a neural network (as seen in previous exercises) and additional arguments for performing cross-validation with multiple executions per fold.

   #### **Function parameters**

   - `topology::AbstractArray{<:Int,1}` — Structure of the neural network.
   - `dataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{<:Any,1}}` — A tuple with:
     - A matrix of inputs (instances in rows),
     - A vector of target labels (categorical). These must be converted into a Boolean matrix via `oneHotEncoding` inside this function.
   - `crossValidationIndices::Array{Int64,1}` — Vector of cross-validation fold assignments for each instance, obtained via the `crossvalidation` function.

   **Optional parameters** (with default values, as specified in the function signature):

   - `numExecutions`: Number of training repetitions per fold.
   - `transferFunctions`: Activation functions for each layer in the ANN.
     ### Question 5.6
     > ❓ Does it make sense to use a linear activation function in hidden layers?
    

`Answer here`

   - `maxEpochs`: Maximum number of training iterations (epochs).
   - `minLoss`: Minimum loss to stop training early.
   - `learningRate`: Learning rate.
   - `validationRatio`: Fraction of data used for validation in early stopping (can be 0).
   - `maxEpochsVal`: Maximum number of epochs without validation improvement before stopping early.


 **Step-by-step implementation**

   1. **Extract class labels**:
      ```julia
      classes = unique(targets)
      ```
   2. **One-hot encode** the categorical target labels using the `oneHotEncoding` function and the computed `classes`.
      ### Question 5.7
      > ❓ What could go wrong if the `classes` vector is not passed explicitly to `oneHotEncoding`?

`Answer here`

   3. **Determine the number of folds** using:
      ```julia
      numFolds = maximum(crossValidationIndices)
      ```
   4. Create one vector per metric (`accuracy`, `error rate`, `sensitivity`, `specificity`, `PPV`, `NPV`, `F1`) to store fold results.

   5. Initialize a **confusion matrix accumulator** (matrix of real numbers with all entries set to 0) for the **global test confusion matrix**.

   **For each fold**:

   - Extract `train` and `test` subsets for **inputs** and **outputs**, based on the cross-validation indices and current fold number.

   - Since ANNs are **non-deterministic**, results from a single training per fold may not be representative.  
     For this reason, train the ANN **multiple times per fold** (as specified in `numExecutions`).

   **Inside each fold**:

   1. Initialize vectors to store the metric results for each execution.  
   2. Create a 3D array of size `(numClasses, numClasses, numExecutions)` to store test confusion matrices from each execution.

   3. For each execution:

      - If `validationRatio > 0`, split the training set into training and validation sets using `holdOut`.  
        > ⚠️ You must adjust the ratio properly, since you’re applying it **on the training subset**, not the full dataset.
        ### Question 5.8  
        > ❓ How can you adapt the validation ratio accordingly?

`Answer here`

      - Train the network using `trainClassANN`.

      - Evaluate it on the test set using `confusionMatrix`.

      - Store the returned metrics and confusion matrix.

   4. After all executions for this fold:

      - Compute the **average** of each metric vector and store it in the global metric vectors.
      - Compute the **mean confusion matrix** using:
        ```julia
        mean(confusionMatrices, dims=3)
        ```
        > Note: This returns a 3D array with one slice; you must extract the 2D matrix from it.

      - Add the resulting matrix to the global confusion matrix.

#### Return values

Once all folds are completed:

- For each metric, compute the **mean and standard deviation** across the folds using `mean(...)` and `std(...)`.
- Return a tuple with 8 elements:
  1. Accuracy (mean, std)
  2. Error rate (mean, std)
  3. Sensitivity (mean, std)
  4. Specificity (mean, std)
  5. PPV (mean, std)
  6. NPV (mean, std)
  7. F1-score (mean, std)
  8. Global test confusion matrix

This function will be called in the next exercise by a general-purpose validation function.  
Unlike ANNs, **other ML models (e.g., SVM, kNN)** are **deterministic** — so they do not need the inner execution loop.  
For them, training once per fold is sufficient to produce stable results.

In [ ]:
function ANNCrossValidation(topology::AbstractArray{<:Int,1},
        dataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{<:Any,1}},
        crossValidationIndices::Array{Int64,1};
        numExecutions::Int=50,
        transferFunctions::AbstractArray{<:Function,1}=fill(σ, length(topology)),
        maxEpochs::Int=1000, minLoss::Real=0.0, learningRate::Real=0.01,
        validationRatio::Real=0, maxEpochsVal::Int=20)
    #TODO